# Tutors Lessons Durations Rates

## Initial setup

In [1]:
from collections import defaultdict

import z3


In [2]:
# Each list represents a dimension of the puzzle, with all its possible values
tutors = ["Baptiste", "Lylian", "Grégoire", "Charles", "Louis"]
lessons = ["anglais", "allemand", "arabe", "latin", "chinois"]
durations = ["30 minutes", "35 minutes", "40 minutes", "45 minutes", "50 minutes"]
rates = ["28€", "35€", "42€", "49€", "56€"]

# Puzzle is a dict of pairs of candidates to boolean variables representing whether the given pair holds true or not
puzzle: defaultdict[str, dict[str, z3.Bool]] = defaultdict(dict)
for t in tutors:
    for l in lessons:
        puzzle[t][l] = z3.Bool(f"{t}_{l}")
    for d in durations:
        puzzle[t][d] = z3.Bool(f"{t}_{d}")
    for r in rates:
        puzzle[t][r] = z3.Bool(f"{t}_{r}")
for r in rates:
    for l in lessons:
        puzzle[r][l] = z3.Bool(f"{r}_{l}")
    for d in durations:
        puzzle[r][d] = z3.Bool(f"{r}_{d}")
for d in durations:
    for l in lessons:
        puzzle[d][l] = z3.Bool(f"{d}_{l}")

# Solver provides an interface to Z3 API
solver = z3.Solver()


## Generic puzzle constraints

Exactly one candidate is valid per line. See <https://stackoverflow.com/questions/43081929/k-out-of-n-constraint-in-z3py>.

In [3]:
for t in tutors:
    solver.add(z3.PbEq([(puzzle[t][l], 1) for l in lessons], k=1))
    solver.add(z3.PbEq([(puzzle[t][d], 1) for d in durations], k=1))
    solver.add(z3.PbEq([(puzzle[t][r], 1) for r in rates], k=1))
for r in rates:
    solver.add(z3.PbEq([(puzzle[r][l], 1) for l in lessons], k=1))
    solver.add(z3.PbEq([(puzzle[r][d], 1) for d in durations], k=1))
for d in durations:
    solver.add(z3.PbEq([(puzzle[d][l], 1) for l in lessons], k=1))


Exactly one candidate is valid per column. See <https://stackoverflow.com/questions/43081929/k-out-of-n-constraint-in-z3py>.

In [4]:
for l in lessons:
    solver.add(z3.PbEq([(puzzle[t][l], 1) for t in tutors], k=1))
    solver.add(z3.PbEq([(puzzle[r][l], 1) for r in rates], k=1))
    solver.add(z3.PbEq([(puzzle[d][l], 1) for d in durations], k=1))
for d in durations:
    solver.add(z3.PbEq([(puzzle[t][d], 1) for t in tutors], k=1))
    solver.add(z3.PbEq([(puzzle[r][d], 1) for r in rates], k=1))
for r in rates:
    solver.add(z3.PbEq([(puzzle[t][r], 1) for t in tutors], k=1))


Lines and columns must match.

In [5]:
for t in tutors:
    for l in lessons:
        for d in durations:
            solver.add(z3.Implies(puzzle[t][l], puzzle[t][d] == puzzle[d][l]))
        for r in rates:
            solver.add(z3.Implies(puzzle[t][l], puzzle[t][r] == puzzle[r][l]))
    for d in durations:
        for r in rates:
            solver.add(z3.Implies(puzzle[t][d], puzzle[t][r] == puzzle[r][d]))
for r in rates:
    for l in lessons:
        for d in durations:
            solver.add(z3.Implies(puzzle[r][l], puzzle[r][d] == puzzle[d][l]))


## Specific puzzle constraints

1. Le cours le plus cher n'a pas duré 35 ni 40 minutes.

In [6]:
solver.add(z3.Not(puzzle["56€"]["35 minutes"]), z3.Not(puzzle["56€"]["40 minutes"]))


2. Le cours de Lylian a duré 5 minutes de moins que celui qui a coûté 35 euros mais plus longtemps que le cours de latin.

In [7]:
solver.add(
    z3.Not(puzzle["Lylian"]["30 minutes"]),
    z3.Implies(puzzle["Lylian"]["35 minutes"], puzzle["35€"]["40 minutes"]),
    z3.Implies(
        puzzle["Lylian"]["35 minutes"],
        z3.Or(
            puzzle["30 minutes"]["latin"],
        ),
    ),
    z3.Implies(puzzle["Lylian"]["40 minutes"], puzzle["35€"]["45 minutes"]),
    z3.Implies(
        puzzle["Lylian"]["40 minutes"],
        z3.Or(
            puzzle["30 minutes"]["latin"],
            puzzle["35 minutes"]["latin"],
        ),
    ),
    z3.Implies(puzzle["Lylian"]["45 minutes"], puzzle["35€"]["50 minutes"]),
    z3.Implies(
        puzzle["Lylian"]["45 minutes"],
        z3.Or(
            puzzle["30 minutes"]["latin"],
            puzzle["35 minutes"]["latin"],
            puzzle["40 minutes"]["latin"],
        ),
    ),
    z3.Not(puzzle["Lylian"]["50 minutes"]),
)


3. Le cours d'arabe a coûté plus cher que celui de Louis mais moins cher que celui de 30 minutes.

In [8]:
solver.add(
    z3.Not(puzzle["28€"]["arabe"]),
    z3.Implies(
        puzzle["35€"]["arabe"],
        z3.Or(
            puzzle["Louis"]["28€"],
        ),
    ),
    z3.Implies(
        puzzle["35€"]["arabe"],
        z3.Or(
            puzzle["42€"]["30 minutes"],
            puzzle["49€"]["30 minutes"],
            puzzle["56€"]["30 minutes"],
        ),
    ),
    z3.Implies(
        puzzle["42€"]["arabe"],
        z3.Or(
            puzzle["Louis"]["28€"],
            puzzle["Louis"]["35€"],
        ),
    ),
    z3.Implies(
        puzzle["42€"]["arabe"],
        z3.Or(
            puzzle["49€"]["30 minutes"],
            puzzle["56€"]["30 minutes"],
        ),
    ),
    z3.Implies(
        puzzle["49€"]["arabe"],
        z3.Or(
            puzzle["Louis"]["28€"],
            puzzle["Louis"]["35€"],
            puzzle["Louis"]["42€"],
        ),
    ),
    z3.Implies(
        puzzle["49€"]["arabe"],
        z3.Or(
            puzzle["56€"]["30 minutes"],
        ),
    ),
    z3.Not(puzzle["56€"]["arabe"]),
)


4. Charles n'enseigne pas l'anglais ni l'arabe mais son cours n'a pas duré 5 minutes de moins que celui de Lylian.

In [9]:
solver.add(
    z3.Not(puzzle["Charles"]["anglais"]),
    z3.Not(puzzle["Charles"]["arabe"]),
    z3.Implies(puzzle["Charles"]["30 minutes"], z3.Not(puzzle["Lylian"]["35 minutes"])),
    z3.Implies(puzzle["Charles"]["35 minutes"], z3.Not(puzzle["Lylian"]["40 minutes"])),
    z3.Implies(puzzle["Charles"]["40 minutes"], z3.Not(puzzle["Lylian"]["45 minutes"])),
    z3.Implies(puzzle["Charles"]["45 minutes"], z3.Not(puzzle["Lylian"]["50 minutes"])),
)


5. Le cours de chinois a duré 5 minutes de moins que celui à 42 euros mais plus longtemps que celui de Baptiste, dont le prix diffère de plus de 7 euros de celui de Charles.

In [10]:
solver.add(
    z3.Not(puzzle["30 minutes"]["chinois"]),
    z3.Implies(puzzle["35 minutes"]["chinois"], puzzle["42€"]["40 minutes"]),
    z3.Implies(
        puzzle["35 minutes"]["chinois"],
        z3.Or(
            puzzle["Baptiste"]["30 minutes"],
        ),
    ),
    z3.Implies(puzzle["40 minutes"]["chinois"], puzzle["42€"]["45 minutes"]),
    z3.Implies(
        puzzle["40 minutes"]["chinois"],
        z3.Or(
            puzzle["Baptiste"]["30 minutes"],
            puzzle["Baptiste"]["35 minutes"],
        ),
    ),
    z3.Implies(puzzle["45 minutes"]["chinois"], puzzle["42€"]["50 minutes"]),
    z3.Implies(
        puzzle["45 minutes"]["chinois"],
        z3.Or(
            puzzle["Baptiste"]["30 minutes"],
            puzzle["Baptiste"]["35 minutes"],
            puzzle["Baptiste"]["40 minutes"],
        ),
    ),
    z3.Not(puzzle["50 minutes"]["chinois"]),
)

solver.add(
    z3.Implies(
        puzzle["Baptiste"]["28€"],
        z3.Or(
            puzzle["Charles"]["42€"],
            puzzle["Charles"]["49€"],
            puzzle["Charles"]["56€"],
        ),
    ),
    z3.Implies(
        puzzle["Baptiste"]["35€"],
        z3.Or(
            puzzle["Charles"]["49€"],
            puzzle["Charles"]["56€"],
        ),
    ),
    z3.Implies(
        puzzle["Baptiste"]["42€"],
        z3.Or(
            puzzle["Charles"]["28€"],
            puzzle["Charles"]["56€"],
        ),
    ),
    z3.Implies(
        puzzle["Baptiste"]["49€"],
        z3.Or(
            puzzle["Charles"]["28€"],
            puzzle["Charles"]["35€"],
        ),
    ),
    z3.Implies(
        puzzle["Baptiste"]["56€"],
        z3.Or(
            puzzle["Charles"]["28€"],
            puzzle["Charles"]["35€"],
            puzzle["Charles"]["42€"],
        ),
    ),
)


6. Le cours de Grégoire a coûté plus cher que le cours d'allemand mais moins cher que le plus long.

In [11]:
solver.add(
    z3.Not(puzzle["Grégoire"]["28€"]),
    z3.Implies(
        puzzle["Grégoire"]["35€"],
        z3.Or(
            puzzle["28€"]["allemand"],
        ),
    ),
    z3.Implies(
        puzzle["Grégoire"]["35€"],
        z3.Or(
            puzzle["42€"]["50 minutes"],
            puzzle["49€"]["50 minutes"],
            puzzle["56€"]["50 minutes"],
        ),
    ),
    z3.Implies(
        puzzle["Grégoire"]["42€"],
        z3.Or(
            puzzle["28€"]["allemand"],
            puzzle["35€"]["allemand"],
        ),
    ),
    z3.Implies(
        puzzle["Grégoire"]["42€"],
        z3.Or(
            puzzle["49€"]["50 minutes"],
            puzzle["56€"]["50 minutes"],
        ),
    ),
    z3.Implies(
        puzzle["Grégoire"]["49€"],
        z3.Or(
            puzzle["28€"]["allemand"],
            puzzle["35€"]["allemand"],
            puzzle["42€"]["allemand"],
        ),
    ),
    z3.Implies(
        puzzle["Grégoire"]["49€"],
        z3.Or(
            puzzle["56€"]["50 minutes"],
        ),
    ),
    z3.Not(puzzle["Grégoire"]["56€"]),
)


## First solution

In [12]:
if solver.check() == z3.sat:
    for t in tutors:
        message = f"{t} donne un cours "
        for l in lessons:
            if solver.model()[puzzle[t][l]]:
                if l[0] in "aeiouy":
                    message += f"d'{l} "
                else:
                    message += f"de {l} "
                break
        for d in durations:
            if solver.model()[puzzle[t][d]]:
                message += f"de {d} "
                break
        for r in rates:
            if solver.model()[puzzle[t][r]]:
                message += f"à {r}."
                break
        print(message)
else:
    print(solver.check())


Baptiste donne un cours d'allemand de 35 minutes à 28€.
Lylian donne un cours d'arabe de 40 minutes à 49€.
Grégoire donne un cours de chinois de 45 minutes à 35€.
Charles donne un cours de latin de 30 minutes à 56€.
Louis donne un cours d'anglais de 50 minutes à 42€.


## Other solutions

Now let's try to find another solution by adding a new constraint. At least one variable must be different from the previous model.

In [13]:
if solver.check() == z3.sat:
    print(
        solver.check(
            z3.Or(
                *[
                    puzzle[t][l] != solver.model()[puzzle[t][l]]
                    for t in tutors
                    for l in lessons
                ],
                *[
                    puzzle[t][d] != solver.model()[puzzle[t][d]]
                    for t in tutors
                    for d in durations
                ],
                *[
                    puzzle[t][r] != solver.model()[puzzle[t][r]]
                    for t in tutors
                    for r in rates
                ],
                *[
                    puzzle[r][l] != solver.model()[puzzle[r][l]]
                    for r in rates
                    for l in lessons
                ],
                *[
                    puzzle[r][d] != solver.model()[puzzle[r][d]]
                    for r in rates
                    for d in durations
                ],
                *[
                    puzzle[d][l] != solver.model()[puzzle[d][l]]
                    for d in durations
                    for l in lessons
                ],
            )
        )
    )
else:
    raise Exception("Cannot find the next solution as no previous solution was found")


unsat
